|||
|---|---|
|Title|nwtimetracking|
|Author|numbworks|
|Version|3.9.0|
||Please check [docs/docs-nwtimetracking.md](../docs/docs-nwtimetracking.md) before proceeding.|

#### Global Modules

In [137]:
from pandas import DataFrame
from typing import Optional

#### Local Modules

In [138]:
from nwtimetracking import SettingBag, DefaultPathProvider, YearProvider, SoftwareProjectNameProvider, TimeTrackingProcessor
from nwtimetracking import ComponentBag, TTMarkdownFactory
from nwpackageversions import LanguageChecker, RequirementChecker, RequirementSummary

#### Language/Requirement Checks

In [139]:
enable_language_status_check : bool = True
enable_requirement_status_check : bool = False

if enable_language_status_check:
    language_status : str = LanguageChecker().get_version_status(required = (3, 12, 5))
    print(language_status)

if enable_requirement_status_check:
    requirement_checker : RequirementChecker = RequirementChecker(logging_function = LambdaCollection().do_nothing_function())
    dockerfile_path : str = requirement_checker.get_default_devcointainer_dockerfile_path()
    requirement_status : Optional[RequirementSummary] = requirement_checker.try_check(file_path = dockerfile_path)
    print(requirement_status)

The installed Python version is matching the expected one (installed: '3.12.5', expected: '3.12.5').


#### Functions : Temp

#### Settings

In [140]:
setting_bag : SettingBag = SettingBag(
    options_tt = ["display"],
    options_tts_by_month = ["display", "save"],
    options_tts_by_year = ["display"],
    options_tts_by_year_month = ["display"],
    options_tts_by_year_month_spnv = ["display"],
    options_tts_by_year_spnv = ["display"],
    options_tts_by_spn = ["display"],
    options_tts_by_spn_spv = ["display"],
    options_tts_by_hashtag = ["display"],
    options_tts_by_hashtag_year = ["display"],
    options_tts_by_efs = ["display"],
    options_tts_by_tr = ["display"],
    options_definitions = ["display"],
    excel_nrows = 1301,
    tts_by_year_month_spnv_display_only_spn = "nwtimetracking",
    tts_by_year_spnv_display_only_spn = "nwtimetracking"
)


#### Main : Analysis

In [141]:
tt_processor : TimeTrackingProcessor = TimeTrackingProcessor(component_bag = ComponentBag(), setting_bag = setting_bag)
tt_processor.initialize()
tt_processor.process_tt()

Date,StartTime,EndTime,Effort,Hashtag,Descriptor,IsSoftwareProject,IsReleaseDay,Year,Month
2024-12-01,15:00,15:45,0h 45m,#python,nwpackageversions v1.8.0,True,True,2024,12
2024-12-01,15:45,16:15,0h 30m,#python,nwreadinglist v4.1.0,True,True,2024,12
2024-12-01,19:15,19:30,0h 15m,#python,nwreadinglist v4.1.0,True,True,2024,12
2024-12-01,19:30,20:15,0h 45m,#python,nwtimetracking v3.9.0,True,True,2024,12
2024-12-01,21:00,23:00,2h 00m,#python,nwtraderaanalytics v4.4.0,True,True,2024,12


In [142]:
tt_processor.process_tts_by_month()


Month,2015,↕,2016,↕,2017,↕,2018,↕,2019,↕,2020,↕,2021,↕,2022,↕,2023,↕,2024
1,00h 00m,↑,18h 00m,↑,88h 30m,↓,80h 15m,↓,60h 00m,↓,29h 15m,↑,53h 00m,↓,00h 00m,↑,06h 00m,↑,45h 45m
2,00h 00m,↑,45h 30m,↑,65h 30m,↑,90h 45m,↓,73h 00m,↓,38h 00m,↓,31h 30m,↓,03h 00m,↑,24h 00m,↑,77h 45m
3,00h 00m,↑,20h 45m,↑,71h 45m,↑,89h 00m,↓,75h 30m,↓,35h 00m,↑,40h 30m,↓,06h 15m,↑,50h 15m,↑,77h 45m
4,00h 00m,↑,37h 30m,↑,68h 00m,↑,88h 30m,↓,59h 45m,↓,40h 45m,↓,19h 00m,↑,27h 30m,↓,19h 00m,↑,29h 30m
5,00h 00m,↑,53h 00m,↑,83h 00m,↑,91h 15m,↓,54h 45m,↓,14h 30m,↑,112h 45m,↓,49h 45m,↓,31h 00m,↑,43h 00m
6,00h 00m,↑,57h 45m,↓,37h 45m,↑,62h 00m,↓,29h 15m,↓,12h 00m,↑,54h 00m,↑,73h 30m,↓,24h 45m,↑,48h 00m
7,00h 00m,↑,46h 45m,↑,65h 30m,↑,69h 30m,↓,24h 15m,↑,34h 00m,↓,23h 30m,↑,51h 00m,↓,16h 30m,↑,67h 00m
8,00h 00m,↑,25h 45m,↑,45h 45m,↑,72h 00m,↓,06h 00m,↑,32h 00m,↑,110h 00m,↓,36h 30m,↑,41h 30m,↓,32h 45m
9,00h 00m,↑,89h 30m,↓,43h 45m,↑,64h 00m,↓,39h 00m,↑,44h 00m,↓,43h 30m,↑,69h 00m,↓,50h 15m,↓,48h 00m
10,08h 00m,↑,82h 15m,↓,64h 30m,↓,46h 45m,↓,45h 30m,↑,48h 00m,↓,35h 30m,↑,38h 30m,↓,20h 00m,↑,101h 30m


This content (id: 'tts_by_month') has been successfully saved as '/home/nwtimetracking/TIMETRACKINGBYMONTH.md'.


In [143]:
tt_processor.process_tts_by_year()


Year,Effort,YearlyTarget,TargetDiff,IsTargetMet
2015,18h 00m,00h 00m,+18h 00m,True
2016,615h 15m,500h 00m,+115h 15m,True
2017,762h 45m,500h 00m,+262h 45m,True
2018,829h 45m,500h 00m,+329h 45m,True
2019,515h 15m,500h 00m,+15h 15m,True
2020,470h 30m,500h 00m,-30h 30m,False
2021,537h 30m,500h 00m,+37h 30m,True
2022,467h 30m,400h 00m,+67h 30m,True
2023,320h 15m,250h 00m,+70h 15m,True
2024,666h 45m,500h 00m,+166h 45m,True


In [144]:
tt_processor.process_tts_by_year_month()


Year,Month,Effort,YearlyTotal,ToTarget
2024,1,45h 45m,45h 45m,-455h 45m
2024,2,77h 45m,123h 30m,-377h 30m
2024,3,77h 45m,201h 15m,-299h 15m
2024,4,29h 30m,230h 45m,-270h 45m
2024,5,43h 00m,273h 45m,-227h 45m
2024,6,48h 00m,321h 45m,-179h 45m
2024,7,67h 00m,388h 45m,-112h 45m
2024,8,32h 45m,421h 30m,-79h 30m
2024,9,48h 00m,469h 30m,-31h 30m
2024,10,101h 30m,571h 00m,+71h 00m


In [145]:
tt_processor.process_tts_by_year_month_spnv()


Year,Month,ProjectName,ProjectVersion,Effort,DME,%_DME,TME,%_TME
2023,9,nwtimetracking,1.0.0,11h 30m,32h 45m,35.11,50h 15m,22.89
2023,10,nwtimetracking,1.0.0,20h 00m,20h 00m,100.00,20h 00m,100.00
2023,11,nwtimetracking,1.1.0,10h 30m,14h 00m,75.00,14h 30m,72.41
2023,11,nwtimetracking,1.2.0,03h 30m,14h 00m,25.00,14h 30m,24.14
2023,12,nwtimetracking,1.3.0,02h 30m,17h 00m,14.71,22h 30m,11.11
2024,1,nwtimetracking,1.3.0,07h 15m,20h 30m,35.37,45h 45m,15.85
2024,1,nwtimetracking,2.0.0,02h 30m,20h 30m,12.20,45h 45m,5.46
2024,2,nwtimetracking,2.2.0,07h 45m,36h 45m,21.09,77h 45m,9.97
2024,3,nwtimetracking,3.0.0,03h 30m,77h 15m,4.53,77h 45m,4.50
2024,5,nwtimetracking,3.2.0,02h 00m,35h 30m,5.63,43h 00m,4.65


In [146]:
tt_processor.process_tts_by_year_spnv()


Year,ProjectName,ProjectVersion,Effort,DYE,%_DYE,TYE,%_TYE
2023,nwtimetracking,1.0.0,31h 30m,213h 00m,14.79,320h 15m,9.84
2023,nwtimetracking,1.1.0,10h 30m,213h 00m,4.93,320h 15m,3.28
2023,nwtimetracking,1.2.0,03h 30m,213h 00m,1.64,320h 15m,1.09
2023,nwtimetracking,1.3.0,02h 30m,213h 00m,1.17,320h 15m,0.78
2024,nwtimetracking,1.3.0,07h 15m,429h 30m,1.69,666h 45m,1.09
2024,nwtimetracking,2.0.0,02h 30m,429h 30m,0.58,666h 45m,0.37
2024,nwtimetracking,2.2.0,07h 45m,429h 30m,1.80,666h 45m,1.16
2024,nwtimetracking,3.0.0,03h 30m,429h 30m,0.81,666h 45m,0.52
2024,nwtimetracking,3.2.0,02h 00m,429h 30m,0.47,666h 45m,0.30
2024,nwtimetracking,3.3.0,01h 00m,429h 30m,0.23,666h 45m,0.15


In [147]:
tt_processor.process_tts_by_spn()


Hashtag,ProjectName,Effort,DE,%_DE,TE,%_TE
#python,nwtraderaanalytics,219h 45m,1874h 30m,11.72,2654h 45m,8.28
#python,nwreadinglist,143h 30m,1874h 30m,7.66,2654h 45m,5.41
#python,nwtimetracking,84h 45m,1874h 30m,4.52,2654h 45m,3.19
#python,nwpackageversions,52h 45m,1874h 30m,2.81,2654h 45m,1.99
#python,nwshared,42h 15m,1874h 30m,2.25,2654h 45m,1.59
#csharp,NW.UnivariateForecasting,208h 00m,1874h 30m,11.10,2654h 45m,7.84
#csharp,NW.NGramTextClassification,207h 30m,1874h 30m,11.07,2654h 45m,7.82
#csharp,NW.MarkdownTables,20h 45m,1874h 30m,1.11,2654h 45m,0.78
#csharp,NW.Shared.Files,05h 30m,1874h 30m,0.29,2654h 45m,0.21
#csharp,NW.Shared.Serialization,04h 15m,1874h 30m,0.23,2654h 45m,0.16


In [148]:
tt_processor.process_tts_by_spn_spv()


ProjectName,ProjectVersion,Effort
NW.MarkdownTables,1.0.0,15h 15m
NW.MarkdownTables,1.0.1,02h 30m
NW.MarkdownTables,3.0.0,03h 00m
NW.NGramTextClassification,1.0.0,73h 15m
NW.NGramTextClassification,1.1.0,07h 30m
NW.NGramTextClassification,2.0.0,15h 30m
NW.NGramTextClassification,3.0.0,28h 00m
NW.NGramTextClassification,3.5.0,39h 45m
NW.NGramTextClassification,3.6.0,17h 30m
NW.NGramTextClassification,3.7.0,08h 00m
